In [ ]:
import numpy as np
import scipy
import time 
import matplotlib.pyplot as plt
from numba import njit, prange
from numpy import empty, float64, ones, zeros, dot, concatenate, add
from numpy.linalg import norm

# Add path to Tensor Fox and load the module.

# import sys
import sys

# read path 
path_tfx = '/home/felipe/PycharmProjects/tensorfox/'

# import module from the path given
sys.path.append(path_tfx)
import TensorFox as tfx
import swamps as sw
import bottlenecks as bn
import Mntensor as mn

In [ ]:
def swimmer():
    T = np.load('swimmer.npy') 
    R = 50
    return T, R

In [ ]:
def hw():
    # Hand writing tensor
    T = np.load('handwritten.npy')        
    R = 150        
    return T, R

In [ ]:
def border_rank():
    T = np.load('border_rank.npy')
    R = 2
    return T, R

In [ ]:
def start_benchmarks(d, d1, d2, d3, r, c, var, trials, options):
    # Create dictionary to gather all results.
    results = {}
    
    for test in ['swimmer', 'hw', 'border_rank', 'mn', 'swamp', 'bottleneck']:
        # Create test tensor.
        if test=='swimmer':
            T, R = swimmer()
        elif test=='hw':
            T, R = hw()
        elif test=='border_rank':
            T, R = border_rank()
        elif test=='mn':
            T, R = mn.Mntensor(d)
        elif test=='swamp':
            T, T_noise = sw.create(d1, d2, d3, r, c, var)
        elif test=='bottleneck':
            T, T_noise = bn.create(d1, d2, d3, r, c, var)   
        
        # Initialize variables.
        m, n, p = T.shape
        Tsize = np.linalg.norm(T) 
        best_error = np.inf
        best_time = np.inf
        print('Testing ' + test)
        
        # Start benchmarks.
        for i in range(trials):
            start = time.time()
            if test in ['swimmer', 'hw', 'border_rank', 'mn']:
                factors, output = tfx.cpd(T, R, options)
                end = time.time() 
                timing = end - start
                rel_error = output.rel_error
            else:
                factors, output = tfx.cpd(T_noise, r, options)
                end = time.time() 
                timing = end - start
                T_approx = tfx.cnv.cpd2tens(factors)
                rel_error = np.linalg.norm(T - T_approx)/Tsize 
            if rel_error < best_error:
                best_error = rel_error
                best_time = timing
                
        class tmp_results:
            error = best_error
            timing = best_time
        results[test] = tmp_results
        
        # Save tensors and results to be tested in Matlab later.
        if test in ['swimmer', 'hw', 'border_rank', 'mn']:
            scipy.io.savemat(test + '.mat', dict(T=T, R=R, tfx_error=best_error))
        else:
            scipy.io.savemat(test + '.mat', dict(T=T, R=r, tfx_error=best_error))
            scipy.io.savemat(test + '_noise.mat', dict(T_noise=T_noise))
        
    return results

In [ ]:
# Example of Some values to run the tests.
d = 5
d1, d2, d3 = 300, 300, 300
r = 15
c = 0.1
var = 0.001
trials = 100
class options:
    tol = 0
    tol_step = 0
    tol_improv = 0
    tol_grad = 0
results = start_benchmarks(d, d1, d2, d3, r, c, var, trials, options)

In [ ]:
# TensorFox's results. 
for test in ['swimmer', 'hw', 'border_rank', 'mn', 'swamp', 'bottleneck']:
    print(test)
    print('    best error =', np.round(results[test].error, 6), '    best time =', np.round(results[test].timing, 3), ' sec')
    print()

In [ ]:
# Before running this cell you must run the script matlab_benchs.m in Matlab (within the same folder).
tensors = ['swimmer', 'hw', 'border_rank', 'mn', 'swamp', 'bottleneck']
i, j = 0, 0
fig, ax = plt.subplots(3, 2, figsize=[12, 10])

for test in tensors:
    algs = ['tfx', 'nls', 'nlsr', 'als', 'alsr', 'minf', 'minfr']
    errors = [results[test].error]
    timings = [results[test].timing]
    for alg in algs[1:]:
        try:
            matlab_result = scipy.io.loadmat(test + '_' + alg + '.mat')
            errors.append(matlab_result['results'][0, 0])
            timings.append(matlab_result['results'][0, 1])
        except FileNotFoundError:
            # This happens when the Matlab solver was unable to find a solution.
            errors.append(np.nan)
            timings.append(np.nan)        
        
    # Sort the results in descending order.
    timings = np.array(timings)
    idx = np.argsort(-timings)
    timings = -np.sort(-timings)
    algs = np.array(algs)
    algs = algs[idx]

    ax[i, j].bar(algs, timings, width=0.3)
    ax[i, j].set_title(test)
    ax[i, j].set_ylabel('seconds')  
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    if j==1:
        j = 0
        i += 1
    else:
        j += 1
        
plt.show()